In [26]:
!pip install transformers

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 2.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/10.4 MB 2.5 MB/s eta 0:00:04
   ------ --------------------------------- 1.6/10.4 MB 2.5 MB/s eta 0:00:04
   --------- ------------------------------ 2.4/10.4 MB 2.6 MB/s eta 0:00:04
   ------------ --------------------------- 3.1/10.4 MB 2.8 MB/s eta 0:00:03
   -------------- ------------------------- 3.7/10.4 MB 2.8 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.4 MB 2.7 MB/s eta 0:00:03
   ------------------ --------------------- 4.7/10.4 MB 2.8 MB/s eta 0:00:03
   ------------------- -------------------- 5.0/10.4 MB 2.6 MB/s eta 0:00:03
   ---------------------- 

In [31]:
!pip install torch

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/212.5 MB 1.4 MB/s eta 0:02:33
   ---------------------------------------- 1.0/212.5 MB 1.7 MB/s eta 0:02:02
   ---------------------------------------- 1.6/212.5 MB 2.0 MB/s eta 0:01:46
   ---------------------------------------- 2.1/212.5 MB 2.1 MB/s eta 0:01:39
   ---------------------------------------- 2.6/212.5 MB 2.3 MB/s eta 0:01:34
    --------------------------------------- 3.1/212.5 MB 2.3 MB/s eta 0:01:32
    --------------------------------------- 3.7/212.5 MB 2.4 MB/s eta 0:01:29
    --------------------------------------- 4.5/212.5 MB 2.5 MB/s eta 0:01:23
    --------------------------------------- 5.0/212.5 MB 2.5 MB/s eta 0:01:22
   - -------------------------------------- 5.5/212.5 MB 2.6 MB/s eta 0:01:21
   - --

In [35]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 3.9 MB/s eta 0:00:00


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keyboard 
import tensorflow as tf
from tensorflow.keras.models import load_model
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [15]:
model = tf.keras.models.load_model(r"D:\cDrive_Files\GRADUATION_PROJECT\Deployment\LSTM_model.h5")

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("vennify/t5-base-grammar-correction")
model_g = AutoModelForSeq2SeqLM.from_pretrained("vennify/t5-base-grammar-correction")

In [17]:
def correct_grammar(sentence):
    input_text = "grammar: " + sentence
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model_g.generate(inputs, max_length=128, num_beams=5, early_stopping=True)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

In [18]:
# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_keypoints(results):
    """Extract keypoints from MediaPipe results."""
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    return np.concatenate([left_hand, right_hand])

In [22]:
# Grammar tool
# tool = language_tool_python.LanguageToolPublicAPI('en-UK')

actions = np.array(['nice', 'thankyou', 'need', 'meet', 'again', 'signlanguage', 'fine', 'how', 'deaf', 'love', 'what', 'cool', 'mean', 'help', 'happy', 'learn', 'please', 'sad', 'home', 'time', 'name', 'hello', 'can', 'you', 'sign', 'language', 'me', 'your'])

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num%3], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

threshold = 0.8
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Reset detection variables for a fresh start
    sequence = []
    sentence = []
    predictions = []
    grammar_result = ""

    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        # Convert the frame color (BGR to RGB)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Improve performance
        
        # Process the frame
        results = holistic.process(image)
        image.flags.writeable = True  # Make it writable again
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw landmarks
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        # Visualization
            if np.unique(predictions[-15:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3: 
                sentence = sentence[-3:]

            # Check if Enter key is pressed
            if keyboard.is_pressed('enter') and sentence:
                raw_sentence = " ".join(sentence)
                grammar_result = correct_grammar(raw_sentence)

            #sentence_text = grammar_result if grammar_result else " ".join(sentence)

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        display_text = grammar_result if grammar_result else " ".join(sentence)
        cv2.putText(image, display_text, (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Sign Language Recognition', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
home
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
home
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
home
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
home
1/1 ━━━━

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("vennify/t5-base-grammar-correction")
model = AutoModelForSeq2SeqLM.from_pretrained("vennify/t5-base-grammar-correction")

def correct_grammar(text):
    input_text = "grammar: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(inputs, max_length=128, num_beams=5, early_stopping=True)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

sentence = ""
corrected = correct_grammar(sentence)
print(f"Original: {sentence}\nCorrected: {corrected}")

Original: 
Corrected: Then there is the question of whether or not there is a need for it.
